In [ ]:
#Cleaning Jing's Data to be useable with the model

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
import statsmodels.api as sm
import loess
import random
import os
from patsy import dmatrices
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM
from keras.layers import GRU
from keras import regularizers
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import seaborn as sn
import itertools  
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.utils import resample
import matplotlib.pyplot as plt
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
pd.set_option('display.max_columns', None)

Using TensorFlow backend.


In [2]:
years=range(2000,2019)
#Virus range: 60-95 Fahrenheit
#Bin labels for classification
bins={1:"[0,1)",2:"[1,6)", 3:"[6,11)", 4:"[11,16)", 5:"[16,21)", 6:"[21,26)", 7:"[26,31)", 8:"[31,36)", 9:"[36,41)", 10:"[41,46)", 11:"[46,51)", 12:"[51,101)", 13:"[101,151)", 14:"[151,201)", 15:"[201,1000)"}
bins_break=[1,6,11,16,21,26,31,36,41,46,51,101,151,201,1000]
bins_name=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

In [3]:
jing_path="C:/Users/andre/NCSA_Python/SPIN_CDC/jing_lagged_data_init.csv"
county_path="C:/Users/andre/NCSA_Python/SPIN_CDC/cleaned_WNV_Data.csv"
jing_data = pd.read_csv(jing_path)
county_data = pd.read_csv(county_path)
counts=[]
counts=jing_data["count"].tolist()

In [4]:
#classifying one-hot binary
bin_list = []
from bisect import bisect

def grade(count=counts, breakpoints=bins_break, marks=bins_name):
        i = bisect(breakpoints, count)
        return marks[i]

for item in [(grade(score),score) for score in counts ]:
    bin_list.append(item[0])
    
#bin_list

In [5]:
for i in range(1,16):
    jing_data["{}".format(i)]=0

In [6]:
for i in range(0,40391):
    for j in range(1,16):
        if bin_list[i]==j: 
            jing_data["{}".format(j)][i]=1
        else:
            jing_data["{}".format(j)][i]=0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
jing_data.head(655)

,GEOID,year,count,count_lag1,count_lag2,count_lag3,count_lag4,count_lag5,stateAvg_lag1,Incident_nominal_lag1,neigh_death_count_lag1,temp_lag1,prec_lag1,hum_lag1,gini_lag1,County_type_lag1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1001,2006,0,0,0,1,0,0,0.223881,1,0,291.911606,3.166458,75.178524,0.519989,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1001,2007,0,0,0,0,1,0,0.089552,1,6,291.539614,3.900827,75.858843,0.501472,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1001,2008,0,0,0,0,0,1,0.119403,1,0,292.317544,3.087140,71.706429,0.536239,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1001,2009,0,0,0,0,0,0,0.253731,1,11,293.287243,2.213271,67.009978,0.524172,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1001,2010,0,0,0,0,0,0,0.164179,1,0,291.567389,3.681567,72.971491,0.477927,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
650,1101,2006,0,0,2,7,0,0,0.223881,1,0,292.257058,3.256280,75.457933,0.536441,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
651,1101,2007,11,5,0,2,7,0,0.089552,1,1,292.009713,3.761215,75.922392,0.511025,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
652,1101,2008,0,0,5,0,2,7,0.119403,1,0,292.840826,2.690377,71.624671,0.546820,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
653,1101,2009,0,11,0,5,0,2,0.253731,1,0,293.433240,2.674722,68.627887,0.543446,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
counties=county_data[["GEOID","location"]]
geoids=counties.GEOID.unique()
locates=counties.location.unique()

geodict_df=pd.DataFrame()
geodict_df["geoids"]=geoids
geodict_df["locations"]=locates

jing_data_counties = jing_data.merge(geodict_df,left_on="GEOID",right_on="geoids")

del jing_data_counties["geoids"]
jing_data_counties.insert(1, "locations", jing_data_counties.pop("locations"))
jing_data_counties.head
jing_data_counties.to_csv("C:/Users/andre/NCSA_Python/SPIN_CDC/full_jing_data_7_20_2020.csv")

In [10]:
jing_data_counties

,GEOID,locations,year,count,count_lag1,count_lag2,count_lag3,count_lag4,count_lag5,stateAvg_lag1,Incident_nominal_lag1,neigh_death_count_lag1,temp_lag1,prec_lag1,hum_lag1,gini_lag1,County_type_lag1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1001,Alabama-Autauga,2006,0,0,0,1,0,0,0.223881,1,0,291.911606,3.166458,75.178524,0.519989,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1001,Alabama-Autauga,2007,0,0,0,0,1,0,0.089552,1,6,291.539614,3.900827,75.858843,0.501472,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1001,Alabama-Autauga,2008,0,0,0,0,0,1,0.119403,1,0,292.317544,3.087140,71.706429,0.536239,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1001,Alabama-Autauga,2009,0,0,0,0,0,0,0.253731,1,11,293.287243,2.213271,67.009978,0.524172,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1001,Alabama-Autauga,2010,0,0,0,0,0,0,0.164179,1,0,291.567389,3.681567,72.971491,0.477927,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40386,56045,Wyoming-Weston,2014,0,0,0,0,0,0,0.130435,1,5,285.122700,0.925981,45.231379,0.615777,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40387,56045,Wyoming-Weston,2015,0,0,0,0,0,0,0.695652,1,10,282.376653,1.528511,57.823626,0.556435,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40388,56045,Wyoming-Weston,2016,0,0,0,0,0,0,0.000000,1,0,282.187580,1.185288,57.302875,0.517420,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40389,56045,Wyoming-Weston,2017,0,0,0,0,0,0,0.130435,1,2,283.514030,1.335612,55.060926,0.624213,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
cols=list(jing_data_counties)
cols_key = cols[0:4]
cols_val = cols[4:]
jing_data_keys = jing_data_counties[cols_key]
jing_data_val = jing_data_counties[cols_val]
jing_data_keys.to_csv("C:/Users/andre/NCSA_Python/SPIN_CDC/jing_data_keys.csv")
jing_data_val.to_csv("C:/Users/andre/NCSA_Python/SPIN_CDC/jing_data_values.csv")

NameError: name 'jing_data_counties' is not defined

['count_lag1',
 'count_lag2',
 'count_lag3',
 'count_lag4',
 'count_lag5',
 'stateAvg_lag1',
 'Incident_nominal_lag1',
 'neigh_death_count_lag1',
 'temp_lag1',
 'prec_lag1',
 'hum_lag1',
 'gini_lag1',
 'County_type_lag1',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15']

In [5]:
#Cleaning Jing's Testing Data
test_path="C:/Users/andre/NCSA_Python/SPIN_CDC/jing_lagged_submit.csv"
test_data=pd.read_csv(test_path)
test_key=test_data[["GEOID","year"]]
test_key.to_csv("C:/Users/andre/NCSA_Python/SPIN_CDC/jing_test_keys.csv")

del test_data["GEOID"]
del test_data["year"]

test_data.to_csv("C:/Users/andre/NCSA_Python/SPIN_CDC/jing_test_values.csv")

In [7]:
test_key

,GEOID,year
0,1001,2020
1,1003,2020
2,1005,2020
3,1007,2020
4,1009,2020
...,...,...
3102,56037,2020
3103,56039,2020
3104,56041,2020
3105,56043,2020
